## Using Unsloth to finetune

## Install Prerequisite Packages

In [ ]:
# This is necessary for colab
!pip install python-dotenv
!pip install datasets
!pip install plotly
!pip install nbformat
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-0hbirobv/unsloth_7ece29f83129443888b84ec24b54f40b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-0hbirobv/unsloth_7ece29f83129443888b84ec24b54f40b
  Resolved https://github.com/unslothai/unsloth.git to commit a2f8db3e7341f983af5814a2c56f54fa29ee548d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.10.7-py3-none-any.whl size=164377 sha256=c8373ee7d1e549f3d79ee9a9ed4aed159bd7b745ec8366ef8f6c71758fc31eb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-med30h0b/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2024.10.6
    Uninstalling unsloth-2024.10.6:
      Successfully uninstalled unsloth-2024.10.6


## Load `.env`

In [2]:
import os
import sys

from datasets import Dataset

from dotenv import find_dotenv, load_dotenv

load_dotenv()

/home/owen/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## Important Global Parameters

In [3]:
FINETUNING_DATASET_NAME="CPSC532/2024NOV2_arxiv_qa_data_cleaned"
OUTPUT_MODEL_NAME="2024NOV02_cleaned_arxiv_qa_data"
BASE_MODEL_NAME="unsloth/Llama-3.2-3B-Instruct"

## API Keys

In [4]:
# Could also insert the token here directly
HF_TOKEN = os.getenv("HUGGINGFACE_API_KEY")

Leveraging Unsloth notebooks for finetuning

In [5]:
max_seq_length = 16000 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


In [6]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL_NAME, # or choose "unsloth/Llama-3.2-1B-Instruct"
    # model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3080 Ti. Max memory: 11.753 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset


## Get dataset

In [9]:
dataset_finetune = load_dataset(
    FINETUNING_DATASET_NAME,
    split="train",
    token=HF_TOKEN
)

In [10]:
dataset_finetune

Dataset({
    features: ['filename', 'source', 'source_type', 'chunk', 'question', 'answer'],
    num_rows: 681
})

In [11]:

dataset_finetune['question'][0]

'What are some common challenges faced by healthcare professionals during doctor-patient interactions in Puskesmas settings?'

In [12]:

dataset_finetune['answer'][0]

'According to the paper "Using LLM for Real-Time Transcription and Summarization of Doctor-Patient Interactions into ePuskesmas in Indonesia", the primary issue contributing to inefficiency in Puskesmas is the time-consuming nature of doctor-patient interactions. Doctors are required to conduct thorough consultations, which involve diagnosing the patient\'s condition, providing treatment advice, and often recording detailed notes into electronic health records (EHR). This process can be prolonged due to various factors, including the need for doctors to ask clarifying questions, especially in regions with diverse linguistic backgrounds.\n\nThe paper highlights that while the diagnosing task is critical for doctors, the transcription and summarization steps can often be automated using AI technologies to improve the time efficiency of the interactions and help doctors improve quality of care and optimize opportunities for early diagnosis and intervention.'

## Convert dataset to messages format

In [13]:
def convert_to_messages_format(example):
    return [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answer']},
    ]

In [14]:
dataset_finetune = dataset_finetune.map(
    lambda x: {
        'conversations' : convert_to_messages_format(x)
        }
)

In [15]:
dataset_finetune = dataset_finetune.map(formatting_prompts_func, batched = True,)

In [16]:
dataset_finetune['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are some common challenges faced by healthcare professionals during doctor-patient interactions in Puskesmas settings?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAccording to the paper "Using LLM for Real-Time Transcription and Summarization of Doctor-Patient Interactions into ePuskesmas in Indonesia", the primary issue contributing to inefficiency in Puskesmas is the time-consuming nature of doctor-patient interactions. Doctors are required to conduct thorough consultations, which involve diagnosing the patient\'s condition, providing treatment advice, and often recording detailed notes into electronic health records (EHR). This process can be prolonged due to various factors, including the need for doctors to ask clarifying questions, especially in regions with diverse linguistic ba

## Set Training Parameters

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_finetune,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,  # Affects memory usage
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1, # Affects memory usage
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. Look into this

In [18]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [19]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat are some common challenges faced by healthcare professionals during doctor-patient interactions in Puskesmas settings?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAccording to the paper "Using LLM for Real-Time Transcription and Summarization of Doctor-Patient Interactions into ePuskesmas in Indonesia", the primary issue contributing to inefficiency in Puskesmas is the time-consuming nature of doctor-patient interactions. Doctors are required to conduct thorough consultations, which involve diagnosing the patient\'s condition, providing treatment advice, and often recording detailed notes into electronic health records (EHR). This process can be prolonged due to various factors, including the need for doctors to ask clarifying questions, especially in regions with diverse linguistic ba

In [20]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                    \n\nAccording to the paper, the manual entry of information into the ePuskesmas application turns it into a labor-intensive and time-consuming task for healthcare providers because they still have to maintain manual records as a copy. This added workload and time required to input data into the ePuskesmas application decreases the overall efficiency (Section I).\n\nIn other words, healthcare providers are already burdened with maintaining paper-based records in addition to entering information into the electronic health record system, which increases their workload and reduces their efficiency.\n\nSource: [8] B. Oktora and S. T. Putri, “The effectiveness of e-Puskesmas application on employee performance in Sindang Barang Puskesmas Bogor: Efektifitas aplikasi e-Puskesmas terhadap kinerja pegawai di Puskesmas Sindang Barang Kota Bogor,” Jurnal Ilmiah Wijaya, vol. 10, no. 2, pp. 44–49, 2018.\n\nReference [8] is cited in

In [21]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3080 Ti. Max memory = 11.753 GB.
3.275 GB of memory reserved.


## Train

In [22]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 681 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 1,700
 "-____-"     Number of trainable parameters = 194,510,848


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


  0%|          | 1/1700 [00:01<52:41,  1.86s/it]

{'loss': 1.2077, 'grad_norm': 1.6806998252868652, 'learning_rate': 4e-05, 'epoch': 0.01}


  0%|          | 2/1700 [00:03<40:54,  1.45s/it]

...

{'loss': 0.0088, 'grad_norm': 0.08964187651872635, 'learning_rate': 1.1799410029498526e-07, 'epoch': 9.98}


100%|██████████| 1700/1700 [36:54<00:00,  1.19s/it]

{'loss': 0.0007, 'grad_norm': 0.008786909282207489, 'learning_rate': 0.0, 'epoch': 9.99}


100%|██████████| 1700/1700 [36:56<00:00,  1.30s/it]

{'train_runtime': 2216.4283, 'train_samples_per_second': 3.073, 'train_steps_per_second': 0.767, 'train_loss': 0.2001394452722161, 'epoch': 9.99}


In [23]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2216.4283 seconds used for training.
36.94 minutes used for training.
Peak reserved memory = 8.201 GB.
Peak reserved memory for training = 4.926 GB.
Peak reserved memory % of max memory = 69.778 %.
Peak reserved memory for training % of max memory = 41.913 %.


In [29]:
import pandas as pd
import matplotlib.pyplot as plt
training_df = pd.DataFrame(trainer.state.log_history)

In [32]:
training_df.head(2)

,loss,grad_norm,learning_rate,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,1.2077,1.680700,0.00004,0.005874,1,NaN,NaN,NaN,NaN,NaN
1,1.3629,3.645731,0.00008,0.011747,2,NaN,NaN,NaN,NaN,NaN


In [47]:
from helpers import create_training_plots

In [49]:
fig = create_training_plots(training_df)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [40]:
training_df.to_csv(f"training_logs/{OUTPUT_MODEL_NAME}.csv", index = False)

## Run Inference

In [41]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def get_response(user_query):
    messages = [
    {"role": "user", "content": user_query},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                            temperature = 1.5, min_p = 0.1)
    return tokenizer.batch_decode(outputs)

In [42]:
dataset_finetune['question'][0]

'What are some common challenges faced by healthcare professionals during doctor-patient interactions in Puskesmas settings?'

Need to investigate how changing the question affects responses

In [43]:
resp = get_response(dataset_finetune['question'][0])
print(resp[0].split("<|start_header_id|>assistant<|end_header_id|>")[1])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




According to the paper "Using LLM for Real-Time Transcription and Summarization of Doctor-Patient Interactions into ePuskesmas in Indonesia", the primary issue contributing to inefficiency in Puskesmas is the time-consuming nature of doctor-patient interactions. Doctors are required to conduct thorough consultations, which


## Save to HF

In [44]:
print(f"Model dtype: {next(model.parameters()).dtype}")


Model dtype: torch.bfloat16


In [ ]:
model.push_to_hub_gguf(
        f"CPSC532/{OUTPUT_MODEL_NAME}",
        tokenizer,
        # quantization_method = ["q4_k_m", "q8_0", "q5_k_m"], 
        quantization_method = ["not_quantized"], # save original precision (bfloat16)
        token = HF_TOKEN
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 32.94 out of 62.67 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 90.30it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['bf16'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at CPSC532/2024NOV02_cleaned_arxiv_qa_data into bf16 GGUF format.
The output location will be /home/owen/Desktop/github/532/implementation/finetuning/CPSC532/2024NOV02_cleaned_arxiv_qa_data/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: 2024NOV02_cleaned_arxiv_qa_data
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.floa

unsloth.BF16.gguf: 100%|██████████| 6.43G/6.43G [04:00<00:00, 26.8MB/s]
No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/CPSC532/2024NOV02_cleaned_arxiv_qa_data


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/CPSC532/2024NOV02_cleaned_arxiv_qa_data
